In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Prerequisite

In [ ]:
!pip install transformers

## Read Input file (.csv -> DataFrame)

In [ ]:
import pandas as pd

input_df = pd.read_csv('../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

In [ ]:
input_df.head(5)

In [ ]:
input_df.describe()

## Split train & test

In [ ]:
def read_spam(target_df):
    texts = []
    labels = []
    for label_dir in ["ham", "spam"]:
        for row_idx in range(len(target_df)):
            labels.append(0 if target_df.iloc[row_idx][0] == 'ham' else 1) # 0: 'ham', 1: 'spam'
            texts.append(target_df.iloc[row_idx][1])
    return texts, labels

input_texts, input_labels = read_spam(input_df)

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(input_texts, input_labels, test_size=.2)

## DistilBert

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

## SpamDataset

In [ ]:
import torch

class SPAMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = SPAMDataset(train_encodings, train_labels)
val_dataset = SPAMDataset(val_encodings, val_labels)

## Fine-tuning

In [ ]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW
from tqdm import tqdm, notebook

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

## Evaluation

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

for i, batch in enumerate(val_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    if i % 5 == 0:
        print(f"Validation [{i}] loss: {loss:.5f}")